In [75]:
# Connect to the Google Account

import subprocess
# Install the Google Cloud SDK
subprocess.run(["gcloud", "auth", "application-default", "login"])

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=U1B2yUUWGX6jGJmozZYgJ0L8gTyjuD&access_type=offline&code_challenge=PEy8VUW5PLEgFSRmRjQ9tiX3-WLFFZ3wA5xHoOLrpgA&code_challenge_method=S256


Credentials saved to file: [/Users/zhuyu/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "carbon-virtue-378402" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


CompletedProcess(args=['gcloud', 'auth', 'application-default', 'login'], returncode=0)

In [76]:
from google.cloud import bigquery
import pandas as pd
import os

# Initialize BigQuery client
client = bigquery.Client()

# Function to run queries and return DataFrame
def run_query(query):
    return client.query(query).to_dataframe()

In [77]:
# 1. Get basic patient info (admissions + demographics (excluding marital_status) + death status)
patient_info_query = """
SELECT adm.subject_id, adm.hadm_id, adm.admittime, adm.dischtime, 
       pat.gender, pat.anchor_age AS approximate_age_at_admission, 
       adm.race, adm.hospital_expire_flag,
       DATETIME_DIFF(adm.admittime, DATETIME(pat.anchor_year, 1, 1, 0, 0, 0), YEAR) + pat.anchor_age AS actual_age
FROM `physionet-data.mimiciv_hosp.admissions` AS adm
JOIN `physionet-data.mimiciv_hosp.patients` AS pat
ON adm.subject_id = pat.subject_id
WHERE adm.admittime IS NOT NULL
ORDER BY subject_id
"""
patient_info_df = run_query(patient_info_query)

# Set in-hospital death status
patient_info_df['in_hospital_death'] = patient_info_df['hospital_expire_flag'] == 1
patient_info_df = patient_info_df.drop(columns=['hospital_expire_flag'])

# Calculate length of stay and keep data with positive L.O.S
patient_info_df['admittime'] = pd.to_datetime(patient_info_df['admittime'])
patient_info_df['dischtime'] = pd.to_datetime(patient_info_df['dischtime'])
patient_info_df['length_of_stay'] = (patient_info_df['dischtime'] - patient_info_df['admittime']).dt.total_seconds() / (60 * 60 * 24)
patient_info_df = patient_info_df[patient_info_df['length_of_stay'] > 0]

# Display a sample of the resulting DataFrame
print(patient_info_df.head())
print(f"Number of rows in the dataset: {patient_info_df.shape[0]}")

/Users/zhuyu/anaconda3/envs/COMP90089/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   subject_id   hadm_id           admittime           dischtime gender  \
0    10000032  22841357 2180-06-26 18:27:00 2180-06-27 18:49:00      F   
1    10000032  29079034 2180-07-23 12:35:00 2180-07-25 17:55:00      F   
2    10000032  25742920 2180-08-05 23:44:00 2180-08-07 17:50:00      F   
3    10000032  22595853 2180-05-06 22:23:00 2180-05-07 17:15:00      F   
4    10000068  25022803 2160-03-03 23:16:00 2160-03-04 06:26:00      F   

   approximate_age_at_admission   race  actual_age  in_hospital_death  \
0                            52  WHITE          52              False   
1                            52  WHITE          52              False   
2                            52  WHITE          52              False   
3                            52  WHITE          52              False   
4                            19  WHITE          19              False   

   length_of_stay  
0        1.015278  
1        2.222222  
2        1.754167  
3        0.786111  
4        0.29861

In [78]:
patient_info_df

,subject_id,hadm_id,admittime,dischtime,gender,approximate_age_at_admission,race,actual_age,in_hospital_death,length_of_stay
0,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,F,52,WHITE,52,False,1.015278
1,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,F,52,WHITE,52,False,2.222222
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,F,52,WHITE,52,False,1.754167
3,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,F,52,WHITE,52,False,0.786111
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,F,19,WHITE,19,False,0.298611
...,...,...,...,...,...,...,...,...,...,...
431226,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,F,46,WHITE,46,False,17.074306
431227,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,F,46,WHITE,48,False,10.011111
431228,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,M,58,WHITE,58,False,3.491667
431229,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58,WHITE,58,True,6.996528


In [79]:
# Count number of patients
num_patients = patient_info_df['subject_id'].nunique()
print(f"Number of patients: {num_patients}")

Number of patients: 180677


In [80]:
# 2. Get high lipase level patients

# Step 1: Retrieve item IDs for lipase tests in ICU
lipase_item_query_icu = """
SELECT itemid, label, category
FROM physionet-data.mimiciv_icu.d_items
WHERE LOWER(label) LIKE '%lipase%'
"""
lipase_items_df_icu = run_query(lipase_item_query_icu)
print(lipase_items_df_icu)
lipase_itemids_icu = "225672"  # Lipase item ID for ICU

# Step 2: Retrieve item IDs for lipase tests in hosp
lipase_item_query_hosp = """
SELECT itemid, label, fluid
FROM physionet-data.mimiciv_hosp.d_labitems
WHERE LOWER(label) LIKE '%lipase%'
"""
lipase_items_df_hosp = run_query(lipase_item_query_hosp)
print(lipase_items_df_hosp)
lipase_items_hosp = [50956, 50844, 51055, 51036]  # Lipase item IDs for hosp

# Step 3: Retrieve lipase values from ICU
lipase_values_query_icu = f"""
SELECT subject_id, hadm_id, charttime, valuenum AS lipase_level
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid IN ({lipase_itemids_icu})
ORDER BY subject_id, charttime
"""
lipase_values_df_icu = run_query(lipase_values_query_icu)

# Step 4: Retrieve lipase values from hosp
lipase_values_query_hosp = f"""
SELECT subject_id, hadm_id, charttime, valuenum AS lipase_level
FROM `physionet-data.mimiciv_hosp.labevents`
WHERE itemid IN ({', '.join(map(str, lipase_items_hosp))})
ORDER BY subject_id, charttime
"""
lipase_values_df_hosp = run_query(lipase_values_query_hosp)

# Step 5: Combine ICU and hosp lipase records
lipase_values_df = pd.concat([lipase_values_df_icu, lipase_values_df_hosp], ignore_index=True)

# Step 6: Merge with patient info to add age and other basic details (only one merge needed)
lipase_values_df = pd.merge(
    lipase_values_df,
    patient_info_df.copy(),
    on=['subject_id', 'hadm_id'],
    how='left'
)

# Step 7: Define a function to check high lipase levels based on age threshold
def check_high_lipase(row, age_cutoff=60):
    # Check if 'actual_age' is NaN and skip if so
    if pd.isna(row['actual_age']):
        return False  # or set a default value, depending on the requirement
    
    upper_limit = 140 if row['actual_age'] < age_cutoff else 151
    return row['lipase_level'] >= 3 * upper_limit

# Step 8: Filter to include only records with high lipase levels
high_lipase_df = lipase_values_df[lipase_values_df.apply(check_high_lipase, axis=1)].copy()


# Step 9: Sort and retain the earliest record for each subject_id and hadm_id combination
high_lipase_df = high_lipase_df.sort_values(by=['subject_id', 'hadm_id', 'charttime']).drop_duplicates(
    subset=['subject_id', 'hadm_id'], keep='first'
)


/Users/zhuyu/anaconda3/envs/COMP90089/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   itemid   label category
0  225672  Lipase     Labs


/Users/zhuyu/anaconda3/envs/COMP90089/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   itemid               label             fluid
0   50956              Lipase             Blood
1   50844     Lipase, Ascites           Ascites
2   51055     Lipase, Pleural           Pleural
3   51036  Lipase, Body Fluid  Other Body Fluid


/Users/zhuyu/anaconda3/envs/COMP90089/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [81]:
# Display results
print(high_lipase_df.head())
print(f"Number of high lipase cases: {high_lipase_df.shape[0]}")
print(f"Number of unique patients with high lipase levels: {high_lipase_df['subject_id'].nunique()}")

       subject_id   hadm_id           charttime  lipase_level  \
1        10004606  29242151 2159-02-20 18:30:00        1222.0   
17547    10006431  24638489 2129-01-23 23:36:00         508.0   
15       10017531  22580355 2159-09-22 20:56:00        1164.0   
17804    10021357  25937617 2144-12-30 06:55:00        1249.0   
55       10036086  28728587 2196-05-26 09:25:00         677.0   

                admittime           dischtime gender  \
1     2159-02-20 13:43:00 2159-03-06 16:51:00      F   
17547 2129-01-24 01:08:00 2129-01-30 16:50:00      F   
15    2159-09-22 19:30:00 2159-10-24 13:40:00      M   
17804 2144-12-27 19:41:00 2145-01-04 19:54:00      F   
55    2196-05-20 02:47:00 2196-06-12 11:42:00      M   

       approximate_age_at_admission   race  actual_age  in_hospital_death  \
1                                64  WHITE          64              False   
17547                            66  WHITE          67              False   
15                               63  WHIT

In [82]:
# Generate a CSV file with the high lipase cases
high_lipase_df.to_csv('high_lipase_cases.csv', index=False)

In [83]:
# # 3. Get high amylase level patients

# # Step 1: Retrieve Amylase Item IDs for both hosp and ICU
# amylase_item_query_hosp = """
# SELECT itemid, label, fluid
# FROM `physionet-data.mimiciv_hosp.d_labitems`
# WHERE LOWER(label) LIKE '%amylase%'
# """
# amylase_items_df_hosp = run_query(amylase_item_query_hosp)
# print("Amylase Items in hosp:")
# print(amylase_items_df_hosp)

# amylase_item_query_icu = """
# SELECT itemid, label, category
# FROM `physionet-data.mimiciv_icu.d_items`
# WHERE LOWER(label) LIKE '%amylase%'
# """
# amylase_items_df_icu = run_query(amylase_item_query_icu)
# print("Amylase Items in icu:")
# print(amylase_items_df_icu)

# # Identified Amylase item IDs for both hosp and ICU
# amylase_itemids = [50867, 53087, 51964, 220581]

# # Step 2: Retrieve Amylase Values from hosp
# amylase_values_query_hosp = f"""
# SELECT subject_id, hadm_id, charttime, valuenum AS amylase_level
# FROM `physionet-data.mimiciv_hosp.labevents`
# WHERE itemid IN ({', '.join(map(str, amylase_itemids))})
# ORDER BY subject_id, charttime
# """
# amylase_values_df_hosp = run_query(amylase_values_query_hosp)

# # Step 3: Retrieve Amylase Values from ICU
# amylase_values_query_icu = f"""
# SELECT subject_id, hadm_id, charttime, valuenum AS amylase_level
# FROM `physionet-data.mimiciv_icu.chartevents`
# WHERE itemid IN ({', '.join(map(str, amylase_itemids))})
# ORDER BY subject_id, charttime
# """
# amylase_values_df_icu = run_query(amylase_values_query_icu)

# # Step 4: Concatenate ICU and hosp data
# amylase_values_df = pd.concat([amylase_values_df_hosp, amylase_values_df_icu], ignore_index=True)

# # Step 5: Filter to include only records with amylase levels > 1000
# amylase_critical_df = amylase_values_df[amylase_values_df['amylase_level'] > 1000]

# # Step 6: Sort and retain the earliest record for each subject_id and hadm_id
# amylase_critical_df = amylase_critical_df.sort_values(by=['subject_id', 'hadm_id', 'charttime']).drop_duplicates(
#     subset=['subject_id', 'hadm_id'], keep='first'
# )

# # Count number of unique patients with high amylase levels
# num_patients = amylase_critical_df['subject_id'].nunique()
# print(amylase_critical_df.head())
# print(f"Number of high amylase cases: {amylase_critical_df.shape[0]}")
# print(f"Number of unique patients with high amylase levels: {num_patients}")

# # Step 7: Inner join high Amylase level records with high Lipase level records
# hl_ha_df = pd.merge(
#     high_lipase_df.copy(),
#     amylase_critical_df[['subject_id', 'hadm_id', 'amylase_level']],
#     on=['subject_id', 'hadm_id'],
#     how='outer',
#     suffixes=('_lipase', '_amylase')
# )


In [84]:
# # Display the combined dataset
# print(hl_ha_df.head())
# print(f"Number of rows in the combined dataset with high lipase or high amylase levels: {hl_ha_df.shape[0]}")
# print(f"Number of unique patients in the combined dataset with high lipase or high amylase levels: {hl_ha_df['subject_id'].nunique()}")

In [85]:
# # 4. Get high CRP level patients
# # Step 1: Retrieve CRP Item IDs for both hosp and ICU
# crp_item_query_hosp = """
# SELECT itemid, label, fluid
# FROM `physionet-data.mimiciv_hosp.d_labitems`
# WHERE LOWER(label) LIKE '%c-reactive%' or LOWER(label) LIKE '%protein%'
# """
# crp_items_df_hosp = run_query(crp_item_query_hosp)
# print("CRP Items in hosp:")
# print(crp_items_df_hosp)

# crp_item_query_icu = """
# SELECT itemid, label, category
# FROM `physionet-data.mimiciv_icu.d_items`
# WHERE LOWER(label) LIKE '%c-reactive%' or LOWER(label) LIKE '%protein%'
# """
# crp_items_df_icu = run_query(crp_item_query_icu)
# print("CRP Items in icu:")
# print(crp_items_df_icu)

# crp_itemid_hosp = "50889"
# crp_itemid_icu = "51006"

# # Step 2: Retrieve CRP values from hosp
# crp_values_query_hosp = f"""
# SELECT subject_id, hadm_id, charttime, valuenum AS crp_level
# FROM `physionet-data.mimiciv_hosp.labevents`
# WHERE itemid IN ({crp_itemid_hosp})
# ORDER BY subject_id, charttime
# """
# crp_values_df_hosp = run_query(crp_values_query_hosp)

# # Step 3: Retrieve CRP values from icu
# crp_values_query_icu = f"""
# SELECT subject_id, hadm_id, charttime, valuenum AS crp_level
# FROM `physionet-data.mimiciv_icu.chartevents`
# WHERE itemid IN ({crp_itemid_icu})
# ORDER BY subject_id, charttime
# """
# crp_values_df_icu = run_query(crp_values_query_icu)

# # Step 4: Concatenate ICU and hosp data and filter for first 48 hours after admission
# crp_values_df = pd.concat([crp_values_df_hosp, crp_values_df_icu], ignore_index=True)
# crp_values_df = pd.merge(crp_values_df, patient_info_df[['subject_id', 'hadm_id', 'admittime']], on=['subject_id', 'hadm_id'])

# # Convert timestamps
# crp_values_df['charttime'] = pd.to_datetime(crp_values_df['charttime'])
# crp_values_df['admittime'] = pd.to_datetime(crp_values_df['admittime'])

# # Filter for entries within the first 48 hours after admission
# crp_values_df = crp_values_df[(crp_values_df['charttime'] - crp_values_df['admittime']).dt.total_seconds() / (60 * 60) <= 48]

# # Group by subject_id and hadm_id, and take the maximum CRP level within 48 hours
# crp_values_df = crp_values_df.groupby(['subject_id', 'hadm_id']).agg(
#     crp_level=('crp_level', 'max')
# ).reset_index()

# # Now `crp_values_df` has the maximum CRP levels within the first 48 hours for each patient-admission
# # Step 6: Filter records with CRP > 150 mg/dL (optional, since you may already have this filter)
# crp_critical_df = crp_values_df[crp_values_df['crp_level'] > 150]

# # Count the number of unique patients with high CRP levels
# num_patients = crp_critical_df['subject_id'].nunique()
# print(f"Number of unique patients with high CRP levels within 48 hrs: {num_patients}")

# # Outer join CRP critical data with the existing combined dataset
# hl_hc_df = pd.merge(
#     high_lipase_df.copy(),
#     crp_critical_df[['subject_id', 'hadm_id', 'crp_level']],
#     on=['subject_id', 'hadm_id'],
#     how='outer',
#     suffixes=('', '_crp')
# )

In [86]:
# # Count the rows and unique patients in the high CRP dataset
# crp_critical_df.shape[0]


In [87]:
# hl_hc_df.set_index(['subject_id', 'hadm_id'], inplace=True)
# hl_hc_df.update(patient_info_df.set_index(['subject_id', 'hadm_id']))
# hl_hc_df.reset_index(inplace=True)

In [88]:
# # Filter the dataset to include only rows where both two levels are not NaN
# non_nan_levels_df = hl_hc_df.dropna(subset=['crp_level', 'lipase_level'])

# # Count the number of unique patients with both two levels present
# num_patients_with_all_levels = non_nan_levels_df['subject_id'].nunique()
# print(f"Number of unique patients with both two levels present: {num_patients_with_all_levels}")

# # Count the number of rows with both two levels present
# num_rows_with_all_levels = non_nan_levels_df.shape[0]
# print(f"Number of rows with both two levels present: {num_rows_with_all_levels}")

In [89]:
# 5. Get AP ICD Info
# Step 1: Retrieve records with AP diagnosis based on ICD codes
ap_icd_query = f"""
SELECT subject_id, hadm_id, icd_code, seq_num 
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE icd_code LIKE 'K85%' OR icd_code = '5770'
ORDER BY subject_id, seq_num
"""
ap_icd_df = run_query(ap_icd_query)

# Keep only the first record for each patient
ap_icd_df = ap_icd_df.drop_duplicates(subset=['subject_id', 'hadm_id'], keep='first')

# Step 2: Merge AP diagnosis info with the existing combined dataset
combined_df_with_ap = pd.merge(
    high_lipase_df.copy(),
    ap_icd_df[['subject_id', 'hadm_id', 'icd_code']],
    on=['subject_id', 'hadm_id'],
    how='left'
)

# Step 3: Label records with AP confirmation
combined_df_with_ap['is_confirmed_ap'] = combined_df_with_ap['icd_code'].notna()

# Display the resulting dataset
print(combined_df_with_ap.head())

# Count unique patients confirmed with AP
num_confirmed_ap_patients = combined_df_with_ap[combined_df_with_ap['is_confirmed_ap'] == True]['subject_id'].nunique()
print(f"Number of unique patients with high levels who are confirmed with AP: {num_confirmed_ap_patients}")

# Total number of rows in the combined dataset with AP confirmation
num_rows = combined_df_with_ap.shape[0]
print(f"Number of rows in the combined dataset with AP confirmation: {num_rows}")

/Users/zhuyu/anaconda3/envs/COMP90089/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
1    10006431  24638489 2129-01-23 23:36:00         508.0 2129-01-24 01:08:00   
2    10017531  22580355 2159-09-22 20:56:00        1164.0 2159-09-22 19:30:00   
3    10021357  25937617 2144-12-30 06:55:00        1249.0 2144-12-27 19:41:00   
4    10036086  28728587 2196-05-26 09:25:00         677.0 2196-05-20 02:47:00   

            dischtime gender  approximate_age_at_admission   race  actual_age  \
0 2159-03-06 16:51:00      F                            64  WHITE          64   
1 2129-01-30 16:50:00      F                            66  WHITE          67   
2 2159-10-24 13:40:00      M                            63  WHITE          64   
3 2145-01-04 19:54:00      F                            91  WHITE          91   
4 2196-06-12 11:42:00      M                            57  WHITE          58   

   in_hospital_death  leng

In [90]:
# 6. Get Weight Info
# Step 1: Retrieve weight data for ICU patients (only admit and daily weight)
weight_query = """
SELECT
    subject_id, hadm_id, stay_id, charttime,
    CASE WHEN itemid = 226512 THEN 'admit' ELSE 'daily' END AS weight_type,
    valuenum AS weight
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE valuenum IS NOT NULL
  AND itemid IN (226512, 224639)  -- Admit Weight and Daily Weight
  AND valuenum > 0
ORDER BY subject_id, charttime
"""

# Step 2: Run the query to retrieve weight data
weight_df = run_query(weight_query)

# Step 3: Calculate the average weight per patient (admit and daily weights)
# Group by patient identifiers and calculate the mean weight
average_weight_df = weight_df.groupby(['subject_id', 'hadm_id']).agg(
    average_weight=('weight', 'mean')
).reset_index()

# Step 4: Merge the average weight data with the main combined dataset
combined_df_with_avg_weight = pd.merge(
    combined_df_with_ap.copy(),
    average_weight_df[['subject_id', 'hadm_id', 'average_weight']],
    on=['subject_id', 'hadm_id'],
    how='left'
)

# Display the resulting DataFrame
print(combined_df_with_avg_weight.head())
print(f"Number of rows with average weight data: {combined_df_with_avg_weight.shape[0]}")

/Users/zhuyu/anaconda3/envs/COMP90089/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
1    10006431  24638489 2129-01-23 23:36:00         508.0 2129-01-24 01:08:00   
2    10017531  22580355 2159-09-22 20:56:00        1164.0 2159-09-22 19:30:00   
3    10021357  25937617 2144-12-30 06:55:00        1249.0 2144-12-27 19:41:00   
4    10036086  28728587 2196-05-26 09:25:00         677.0 2196-05-20 02:47:00   

            dischtime gender  approximate_age_at_admission   race  actual_age  \
0 2159-03-06 16:51:00      F                            64  WHITE          64   
1 2129-01-30 16:50:00      F                            66  WHITE          67   
2 2159-10-24 13:40:00      M                            63  WHITE          64   
3 2145-01-04 19:54:00      F                            91  WHITE          91   
4 2196-06-12 11:42:00      M                            57  WHITE          58   

   in_hospital_death  leng

In [91]:
# 7. Get CCI Info
# Step 1: Query for Charlson Comorbidity Index (CCI) based on ICD codes
cci_query = """
WITH diag AS (
    SELECT 
        hadm_id,
        CASE WHEN icd_version = 9 THEN icd_code ELSE NULL END AS icd9_code,
        CASE WHEN icd_version = 10 THEN icd_code ELSE NULL END AS icd10_code
    FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
),
com AS (
    SELECT ad.hadm_id,
        -- Myocardial infarction
        MAX(CASE WHEN SUBSTR(icd9_code, 1, 3) IN ('410','412') OR SUBSTR(icd10_code, 1, 3) IN ('I21','I22') OR SUBSTR(icd10_code, 1, 4) = 'I252' THEN 1 ELSE 0 END) AS myocardial_infarct,
        -- Congestive heart failure
        MAX(CASE WHEN SUBSTR(icd9_code, 1, 3) = '428' OR SUBSTR(icd9_code, 1, 5) IN ('39891','40201','40211','40291','40401','40403','40411','40413','40491','40493') OR SUBSTR(icd9_code, 1, 4) BETWEEN '4254' AND '4259' OR SUBSTR(icd10_code, 1, 3) IN ('I43','I50') OR SUBSTR(icd10_code, 1, 4) IN ('I099','I110','I130','I132','I255','I420','I425','I426','I427','I428','I429','P290') THEN 1 ELSE 0 END) AS congestive_heart_failure,
        -- Peripheral vascular disease
        MAX(CASE WHEN SUBSTR(icd9_code, 1, 3) IN ('440','441') OR SUBSTR(icd9_code, 1, 4) IN ('0930','4373','4471','5571','5579','V434') OR SUBSTR(icd9_code, 1, 4) BETWEEN '4431' AND '4439' OR SUBSTR(icd10_code, 1, 3) IN ('I70','I71') OR SUBSTR(icd10_code, 1, 4) IN ('I731','I738','I739','I771','I790','I792','K551','K558','K559','Z958','Z959') THEN 1 ELSE 0 END) AS peripheral_vascular_disease,
        -- Additional comorbidity definitions (Cerebrovascular disease, Dementia, Chronic pulmonary disease, etc.)
        -- Add other conditions following similar MAX/CASE structure as above for each comorbidity
    FROM `physionet-data.mimiciv_hosp.admissions` ad
    LEFT JOIN diag ON ad.hadm_id = diag.hadm_id
    GROUP BY ad.hadm_id
),
ag AS (
    SELECT 
        hadm_id,
        age,
        CASE WHEN age <= 40 THEN 0 WHEN age <= 50 THEN 1 WHEN age <= 60 THEN 2 WHEN age <= 70 THEN 3 ELSE 4 END AS age_score
    FROM `physionet-data.mimiciv_derived.age`
)
SELECT 
    ad.subject_id,
    ad.hadm_id,
    ag.age_score,
    myocardial_infarct,
    congestive_heart_failure,
    peripheral_vascular_disease,
    -- Include all other comorbidities fields here...
    age_score + myocardial_infarct + congestive_heart_failure + peripheral_vascular_disease
    -- + add all the weighted conditions here as in your full CCI calculation
    AS charlson_comorbidity_index
FROM `physionet-data.mimiciv_hosp.admissions` ad
LEFT JOIN com ON ad.hadm_id = com.hadm_id
LEFT JOIN ag ON com.hadm_id = ag.hadm_id
"""

# Step 2: Run the query to get CCI data
cci_df = run_query(cci_query)

# Step 3: Merge CCI data with the main dataset based on `subject_id` and `hadm_id`
combined_df_with_ap = pd.merge(
    combined_df_with_avg_weight.copy(),
    cci_df[['subject_id', 'hadm_id', 'charlson_comorbidity_index']],
    on=['subject_id', 'hadm_id'],
    how='left'
)

# Step 4: Display the final combined dataset with CCI information
print(combined_df_with_ap.head())
print(f"Number of rows in the combined dataset with CCI: {combined_df_with_ap.shape[0]}")

/Users/zhuyu/anaconda3/envs/COMP90089/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
1    10006431  24638489 2129-01-23 23:36:00         508.0 2129-01-24 01:08:00   
2    10017531  22580355 2159-09-22 20:56:00        1164.0 2159-09-22 19:30:00   
3    10021357  25937617 2144-12-30 06:55:00        1249.0 2144-12-27 19:41:00   
4    10036086  28728587 2196-05-26 09:25:00         677.0 2196-05-20 02:47:00   

            dischtime gender  approximate_age_at_admission   race  actual_age  \
0 2159-03-06 16:51:00      F                            64  WHITE          64   
1 2129-01-30 16:50:00      F                            66  WHITE          67   
2 2159-10-24 13:40:00      M                            63  WHITE          64   
3 2145-01-04 19:54:00      F                            91  WHITE          91   
4 2196-06-12 11:42:00      M                            57  WHITE          58   

   in_hospital_death  leng

In [92]:
# 8. Get Height info
# 1. Get height data for ICU patients (in both inches and centimeters)
height_query = """
WITH ht_in AS (
  SELECT 
    c.subject_id, c.hadm_id, c.stay_id, c.charttime,
    -- Convert height from inches to centimeters
    ROUND(c.valuenum * 2.54, 2) AS height
  FROM `physionet-data.mimiciv_icu.chartevents` c
  WHERE c.valuenum IS NOT NULL
    AND c.itemid = 226707  -- Height in inches
),
ht_cm AS (
  SELECT 
    c.subject_id, c.hadm_id, c.stay_id, c.charttime,
    ROUND(c.valuenum, 2) AS height
  FROM `physionet-data.mimiciv_icu.chartevents` c
  WHERE c.valuenum IS NOT NULL
    AND c.itemid = 226730  -- Height in centimeters
),
-- Merge heights from both inches and centimeters, taking one per charted row
ht_stg0 AS (
  SELECT
    COALESCE(h1.subject_id, h2.subject_id) AS subject_id,
    COALESCE(h1.hadm_id, h2.hadm_id) AS hadm_id,
    COALESCE(h1.stay_id, h2.stay_id) AS stay_id,
    COALESCE(h1.charttime, h2.charttime) AS charttime,
    COALESCE(h1.height, h2.height) AS height
  FROM ht_cm h1
  FULL OUTER JOIN ht_in h2
    ON h1.subject_id = h2.subject_id
    AND h1.hadm_id = h2.hadm_id
    AND h1.charttime = h2.charttime
)
SELECT subject_id, hadm_id, stay_id, charttime, height
FROM ht_stg0
WHERE height IS NOT NULL
  AND height BETWEEN 120 AND 230  -- Filter out unrealistic heights
ORDER BY subject_id, charttime
"""

# Step 2: Run the query to retrieve height data
height_df = run_query(height_query)

# Step 3: Calculate the average height per patient
# Group by patient identifiers and calculate the mean height
average_height_df = height_df.groupby(['subject_id', 'hadm_id']).agg(
    average_height=('height', 'mean')
).reset_index()

# Step 4: Merge the average height data with the main combined dataset
combined_df_with_avg_height = pd.merge(
    combined_df_with_ap.copy(),
    average_height_df[['subject_id', 'hadm_id', 'average_height']],
    on=['subject_id', 'hadm_id'],
    how='left'
)

# Display the resulting DataFrame
print(combined_df_with_avg_height.head())
print(f"Number of rows with average height data: {combined_df_with_avg_height.shape[0]}")

/Users/zhuyu/anaconda3/envs/COMP90089/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
1    10006431  24638489 2129-01-23 23:36:00         508.0 2129-01-24 01:08:00   
2    10017531  22580355 2159-09-22 20:56:00        1164.0 2159-09-22 19:30:00   
3    10021357  25937617 2144-12-30 06:55:00        1249.0 2144-12-27 19:41:00   
4    10036086  28728587 2196-05-26 09:25:00         677.0 2196-05-20 02:47:00   

            dischtime gender  approximate_age_at_admission   race  actual_age  \
0 2159-03-06 16:51:00      F                            64  WHITE          64   
1 2129-01-30 16:50:00      F                            66  WHITE          67   
2 2159-10-24 13:40:00      M                            63  WHITE          64   
3 2145-01-04 19:54:00      F                            91  WHITE          91   
4 2196-06-12 11:42:00      M                            57  WHITE          58   

   in_hospital_death  leng

In [93]:
# # 8. Get vital signs info
# # Step 1: Get the list of unique subject_id and hadm_id
# subject_ids = combined_df_with_ap['subject_id'].dropna().unique().tolist()
# hadm_ids = combined_df_with_ap['hadm_id'].dropna().unique().tolist()

# # Step 2: Define the time limit for the first 24 hours
# time_limit = 24 * 60 * 60  # 24 hours in seconds

# # Step 3: Modify each query to select only the first record within 24 hours of admission

# # 1.1 Respiratory Rate (RR)
# icu_respiratory_rate_query = f"""
# SELECT icu.subject_id, icu.hadm_id, icu.charttime, icu.itemid, icu.valuenum AS respiratory_rate,
# FROM `physionet-data.mimiciv_icu.chartevents` icu
# JOIN `physionet-data.mimiciv_hosp.admissions` adm
# ON icu.subject_id = adm.subject_id AND icu.hadm_id = adm.hadm_id
# WHERE icu.itemid = 220210
# AND TIMESTAMP_DIFF(icu.charttime, adm.admittime, SECOND) <= {time_limit}
# AND icu.subject_id IN ({', '.join(map(str, subject_ids))})
# AND icu.hadm_id IN ({', '.join(map(str, hadm_ids))})
# ORDER BY icu.subject_id, icu.hadm_id, icu.charttime
# """
# icu_respiratory_rate_df = run_query(icu_respiratory_rate_query)
# icu_respiratory_rate_df = icu_respiratory_rate_df.drop_duplicates(subset=['subject_id', 'hadm_id'], keep='first')
# print(icu_respiratory_rate_df)

# # 1.2 Heart Rate (HR)
# icu_heart_rate_query = f"""
# SELECT icu.subject_id, icu.hadm_id, icu.charttime, icu.itemid, icu.valuenum AS heart_rate
# FROM `physionet-data.mimiciv_icu.chartevents` icu
# JOIN `physionet-data.mimiciv_hosp.admissions` adm
# ON icu.subject_id = adm.subject_id AND icu.hadm_id = adm.hadm_id
# WHERE icu.itemid = 220045
# AND TIMESTAMP_DIFF(icu.charttime, adm.admittime, SECOND) <= {time_limit}
# AND icu.subject_id IN ({', '.join(map(str, subject_ids))})
# AND icu.hadm_id IN ({', '.join(map(str, hadm_ids))})
# ORDER BY icu.subject_id, icu.hadm_id, icu.charttime
# """
# icu_heart_rate_df = run_query(icu_heart_rate_query)
# icu_heart_rate_df = icu_heart_rate_df.drop_duplicates(subset=['subject_id', 'hadm_id'], keep='first')
# print(icu_heart_rate_df)

# # 1.3 Oxygen Saturation (OS)
# hosp_oxygen_saturation_query = f"""
# SELECT hosp.subject_id, hosp.hadm_id, hosp.charttime, hosp.itemid, hosp.valuenum AS oxygen_saturation
# FROM `physionet-data.mimiciv_hosp.labevents` hosp
# JOIN `physionet-data.mimiciv_hosp.admissions` adm
# ON hosp.subject_id = adm.subject_id AND hosp.hadm_id = adm.hadm_id
# WHERE hosp.itemid = 50817
# AND TIMESTAMP_DIFF(hosp.charttime, adm.admittime, SECOND) <= {time_limit}
# AND hosp.subject_id IN ({', '.join(map(str, subject_ids))})
# AND hosp.hadm_id IN ({', '.join(map(str, hadm_ids))})
# ORDER BY hosp.subject_id, hosp.hadm_id, hosp.charttime
# """
# hosp_oxygen_saturation_df = run_query(hosp_oxygen_saturation_query)
# hosp_oxygen_saturation_df = hosp_oxygen_saturation_df.drop_duplicates(subset=['subject_id', 'hadm_id'], keep='first')
# print(hosp_oxygen_saturation_df)

# # 1.4 Temperature
# hosp_temperature_query = f"""
# SELECT hosp.subject_id, hosp.hadm_id, hosp.charttime, hosp.itemid, hosp.valuenum AS temperature
# FROM `physionet-data.mimiciv_hosp.labevents` hosp
# JOIN `physionet-data.mimiciv_hosp.admissions` adm
# ON hosp.subject_id = adm.subject_id AND hosp.hadm_id = adm.hadm_id
# WHERE hosp.itemid = 50825
# AND TIMESTAMP_DIFF(hosp.charttime, adm.admittime, SECOND) <= {time_limit}
# AND hosp.subject_id IN ({', '.join(map(str, subject_ids))})
# AND hosp.hadm_id IN ({', '.join(map(str, hadm_ids))})
# ORDER BY hosp.subject_id, hosp.hadm_id, hosp.charttime
# """
# hosp_temperature_df = run_query(hosp_temperature_query)
# hosp_temperature_df = hosp_temperature_df.drop_duplicates(subset=['subject_id', 'hadm_id'], keep='first')
# print(hosp_temperature_df)

# # Step 4: Merge these results with the main dataset
# combined_df_with_vitals = combined_df_with_ap.copy()
# combined_df_with_vitals = pd.merge(combined_df_with_vitals, icu_respiratory_rate_df[['subject_id', 'hadm_id', 'respiratory_rate']], on=['subject_id', 'hadm_id'], how='left')
# combined_df_with_vitals = pd.merge(combined_df_with_vitals, icu_heart_rate_df[['subject_id', 'hadm_id', 'heart_rate']], on=['subject_id', 'hadm_id'], how='left')
# combined_df_with_vitals = pd.merge(combined_df_with_vitals, hosp_oxygen_saturation_df[['subject_id', 'hadm_id', 'oxygen_saturation']], on=['subject_id', 'hadm_id'], how='left')
# combined_df_with_vitals = pd.merge(combined_df_with_vitals, hosp_temperature_df[['subject_id', 'hadm_id', 'temperature']], on=['subject_id', 'hadm_id'], how='left')

# # Display the final combined dataset
# print(combined_df_with_vitals.head())
# print(f"Number of rows in the combined dataset with vital signs: {combined_df_with_vitals.shape[0]}")

In [94]:
# 8. Get vital signs info with '_vital' prefix
# Step 1: Retrieve the list of unique subject_id and hadm_id
subject_ids = combined_df_with_ap['subject_id'].dropna().unique().tolist()
hadm_ids = combined_df_with_ap['hadm_id'].dropna().unique().tolist()


# Step 3: Modify the query to retrieve the first vital sign record within 24 hours of admission
vital_signs_query = f"""
SELECT 
    ce.subject_id,
    ce.hadm_id,
    MIN(ce.charttime) AS first_charttime_vital,
    
    -- Average heart rate within the first record in 24 hours
    AVG(CASE WHEN itemid = 220045 AND valuenum > 0 AND valuenum < 300 THEN valuenum ELSE NULL END) AS heart_rate_vital,
    
    -- Average systolic blood pressure (non-invasive and arterial combined)
    AVG(CASE WHEN itemid IN (220179, 220050) AND valuenum > 0 AND valuenum < 400 THEN valuenum ELSE NULL END) AS sbp_vital,
    
    -- Average diastolic blood pressure (non-invasive and arterial combined)
    AVG(CASE WHEN itemid IN (220180, 220051) AND valuenum > 0 AND valuenum < 300 THEN valuenum ELSE NULL END) AS dbp_vital,
    
    -- Mean arterial pressure (non-invasive and arterial combined)
    AVG(CASE WHEN itemid IN (220052, 220181, 225312) AND valuenum > 0 AND valuenum < 300 THEN valuenum ELSE NULL END) AS mbp_vital,
    
    -- Average respiratory rate
    AVG(CASE WHEN itemid IN (220210, 224690) AND valuenum > 0 AND valuenum < 70 THEN valuenum ELSE NULL END) AS resp_rate_vital,
    
    -- Temperature in Celsius
    ROUND(
        AVG(
            CASE 
                WHEN itemid = 223761 AND valuenum > 70 AND valuenum < 120 THEN (valuenum - 32) / 1.8 -- Convert Fahrenheit to Celsius
                WHEN itemid = 223762 AND valuenum > 10 AND valuenum < 50 THEN valuenum
                ELSE NULL 
            END
        ), 2) AS temperature_c_vital,
    
    -- Average oxygen saturation (SpO2)
    AVG(CASE WHEN itemid = 220277 AND valuenum > 0 AND valuenum <= 100 THEN valuenum ELSE NULL END) AS spo2_vital
FROM `physionet-data.mimiciv_icu.chartevents` AS ce
JOIN `physionet-data.mimiciv_hosp.admissions` AS adm
ON ce.subject_id = adm.subject_id AND ce.hadm_id = adm.hadm_id
WHERE ce.hadm_id IS NOT NULL
AND ce.itemid IN (
    220045,   -- Heart Rate
    220179,   -- Non-Invasive Blood Pressure Systolic
    220180,   -- Non-Invasive Blood Pressure Diastolic
    220181,   -- Non-Invasive Blood Pressure Mean
    220210,   -- Respiratory Rate
    220050,   -- Arterial Blood Pressure Systolic
    220051,   -- Arterial Blood Pressure Diastolic
    220052,   -- Arterial Blood Pressure Mean
    224690,   -- Respiratory Rate (Total)
    220277,   -- SpO2, peripheral
    223762,   -- Temperature in Celsius
    223761    -- Temperature in Fahrenheit
)
AND ce.subject_id IN ({', '.join(map(str, subject_ids))})
AND ce.hadm_id IN ({', '.join(map(str, hadm_ids))})
GROUP BY ce.subject_id, ce.hadm_id
ORDER BY ce.subject_id, ce.hadm_id, first_charttime_vital
"""

# Execute the query
vital_signs_df = run_query(vital_signs_query)

# Merge the vital signs data with the main dataset and add '_vital' prefix
combined_df_with_vitals = pd.merge(
    combined_df_with_avg_height.copy(),
    vital_signs_df,
    on=['subject_id', 'hadm_id'],
    how='left'
)

# Display sample of the combined dataset
print(combined_df_with_vitals.head())
print(f"Number of rows in the combined dataset with vital signs: {combined_df_with_vitals.shape[0]}")


/Users/zhuyu/anaconda3/envs/COMP90089/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
1    10006431  24638489 2129-01-23 23:36:00         508.0 2129-01-24 01:08:00   
2    10017531  22580355 2159-09-22 20:56:00        1164.0 2159-09-22 19:30:00   
3    10021357  25937617 2144-12-30 06:55:00        1249.0 2144-12-27 19:41:00   
4    10036086  28728587 2196-05-26 09:25:00         677.0 2196-05-20 02:47:00   

            dischtime gender  approximate_age_at_admission   race  actual_age  \
0 2159-03-06 16:51:00      F                            64  WHITE          64   
1 2129-01-30 16:50:00      F                            66  WHITE          67   
2 2159-10-24 13:40:00      M                            63  WHITE          64   
3 2145-01-04 19:54:00      F                            91  WHITE          91   
4 2196-06-12 11:42:00      M                            57  WHITE          58   

   ...  charlson_comorbidi

In [95]:
# 9. Get chemistries info
chemistry_query = f"""
-- Extract chemistry labs for the first measurement after admission
SELECT 
    le.subject_id,
    le.hadm_id,
    MIN(le.charttime) AS first_charttime_chemistry,  -- Record the earliest chart time for each patient
    -- Chemistry measurements with thresholds to exclude outliers
    MAX(CASE WHEN itemid = 50862 AND valuenum <= 10 THEN valuenum ELSE NULL END) AS albumin_chemistry,
    MAX(CASE WHEN itemid = 50930 AND valuenum <= 10 THEN valuenum ELSE NULL END) AS globulin_chemistry,
    MAX(CASE WHEN itemid = 50976 AND valuenum <= 20 THEN valuenum ELSE NULL END) AS total_protein_chemistry,
    MAX(CASE WHEN itemid = 50868 AND valuenum <= 10000 THEN valuenum ELSE NULL END) AS aniongap_chemistry,
    MAX(CASE WHEN itemid = 50882 AND valuenum <= 10000 THEN valuenum ELSE NULL END) AS bicarbonate_chemistry,
    MAX(CASE WHEN itemid = 51006 AND valuenum <= 300 THEN valuenum ELSE NULL END) AS bun_chemistry,
    MAX(CASE WHEN itemid = 50893 AND valuenum <= 10000 THEN valuenum ELSE NULL END) AS calcium_chemistry,
    MAX(CASE WHEN itemid = 50902 AND valuenum <= 10000 THEN valuenum ELSE NULL END) AS chloride_chemistry,
    MAX(CASE WHEN itemid = 50912 AND valuenum <= 150 THEN valuenum ELSE NULL END) AS creatinine_chemistry,
    MAX(CASE WHEN itemid = 50931 AND valuenum <= 10000 THEN valuenum ELSE NULL END) AS glucose_chemistry,
    MAX(CASE WHEN itemid = 50983 AND valuenum <= 200 THEN valuenum ELSE NULL END) AS sodium_chemistry,
    MAX(CASE WHEN itemid = 50971 AND valuenum <= 30 THEN valuenum ELSE NULL END) AS potassium_chemistry
FROM `physionet-data.mimiciv_hosp.labevents` AS le
JOIN `physionet-data.mimiciv_hosp.admissions` AS adm
ON le.subject_id = adm.subject_id AND le.hadm_id = adm.hadm_id
WHERE le.hadm_id IS NOT NULL
AND le.itemid IN (
    50862, -- Albumin
    50930, -- Globulin
    50976, -- Total Protein
    50868, -- Anion Gap
    50882, -- Bicarbonate
    50893, -- Calcium
    50912, -- Creatinine
    50902, -- Chloride
    50931, -- Glucose
    50971, -- Potassium
    50983, -- Sodium
    51006  -- Urea Nitrogen (BUN)
)
AND le.subject_id IN ({', '.join(map(str, subject_ids))})
AND le.hadm_id IN ({', '.join(map(str, hadm_ids))})
GROUP BY le.subject_id, le.hadm_id
ORDER BY le.subject_id, le.hadm_id, first_charttime_chemistry
"""

# Execute the chemistry query
chemistry_df = run_query(chemistry_query)

# Merge the chemistry data with the main dataset containing vital signs and patient info
combined_df_with_chemistry = pd.merge(
    combined_df_with_vitals.copy(),
    chemistry_df,
    on=['subject_id', 'hadm_id'],
    how='left'
)

# Display sample of the combined dataset
print(combined_df_with_chemistry.head())
print(f"Number of rows in the combined dataset with chemistry info: {combined_df_with_chemistry.shape[0]}")


/Users/zhuyu/anaconda3/envs/COMP90089/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
1    10006431  24638489 2129-01-23 23:36:00         508.0 2129-01-24 01:08:00   
2    10017531  22580355 2159-09-22 20:56:00        1164.0 2159-09-22 19:30:00   
3    10021357  25937617 2144-12-30 06:55:00        1249.0 2144-12-27 19:41:00   
4    10036086  28728587 2196-05-26 09:25:00         677.0 2196-05-20 02:47:00   

            dischtime gender  approximate_age_at_admission   race  actual_age  \
0 2159-03-06 16:51:00      F                            64  WHITE          64   
1 2129-01-30 16:50:00      F                            66  WHITE          67   
2 2159-10-24 13:40:00      M                            63  WHITE          64   
3 2145-01-04 19:54:00      F                            91  WHITE          91   
4 2196-06-12 11:42:00      M                            57  WHITE          58   

   ...  total_protein_chem

In [115]:
# 10. Get enzymes info
enzyme_query = f"""
-- Extract enzyme labs for the first measurement after admission
SELECT 
    le.subject_id,
    le.hadm_id,
    MIN(le.charttime) AS first_charttime_enzyme,  -- Record the earliest chart time for each patient
    -- Enzyme measurements with thresholds to exclude outliers
    MAX(CASE WHEN itemid = 50861 AND valuenum > 0 THEN valuenum ELSE NULL END) AS alt_enzyme,
    MAX(CASE WHEN itemid = 50863 AND valuenum > 0 THEN valuenum ELSE NULL END) AS alp_enzyme,
    MAX(CASE WHEN itemid = 50878 AND valuenum > 0 THEN valuenum ELSE NULL END) AS ast_enzyme,
    MAX(CASE WHEN itemid = 50867 AND valuenum > 0 THEN valuenum ELSE NULL END) AS amylase_enzyme,
    MAX(CASE WHEN itemid = 50885 AND valuenum > 0 THEN valuenum ELSE NULL END) AS bilirubin_total_enzyme,
    MAX(CASE WHEN itemid = 50883 AND valuenum > 0 THEN valuenum ELSE NULL END) AS bilirubin_direct_enzyme,
    MAX(CASE WHEN itemid = 50884 AND valuenum > 0 THEN valuenum ELSE NULL END) AS bilirubin_indirect_enzyme,
    MAX(CASE WHEN itemid = 50910 AND valuenum > 0 THEN valuenum ELSE NULL END) AS ck_cpk_enzyme,
    MAX(CASE WHEN itemid = 50911 AND valuenum > 0 THEN valuenum ELSE NULL END) AS ck_mb_enzyme,
    MAX(CASE WHEN itemid = 50927 AND valuenum > 0 THEN valuenum ELSE NULL END) AS ggt_enzyme,
    MAX(CASE WHEN itemid = 50954 AND valuenum > 0 THEN valuenum ELSE NULL END) AS ld_ldh_enzyme
FROM `physionet-data.mimiciv_hosp.labevents` AS le
JOIN `physionet-data.mimiciv_hosp.admissions` AS adm
ON le.subject_id = adm.subject_id AND le.hadm_id = adm.hadm_id
WHERE le.hadm_id IS NOT NULL
AND le.itemid IN (
    50861, -- Alanine transaminase (ALT)
    50863, -- Alkaline phosphatase (ALP)
    50878, -- Aspartate transaminase (AST)
    50867, -- Amylase
    50885, -- Total Bilirubin
    50884, -- Indirect Bilirubin
    50883, -- Direct Bilirubin
    50910, -- CK-CPK
    50911, -- CK-MB
    50927, -- Gamma Glutamyltransferase (GGT)
    50954  -- LD-LDH
)
AND le.subject_id IN ({', '.join(map(str, subject_ids))})
AND le.hadm_id IN ({', '.join(map(str, hadm_ids))})
GROUP BY le.subject_id, le.hadm_id
ORDER BY le.subject_id, le.hadm_id, first_charttime_enzyme
"""

# Execute the enzyme query
enzyme_df = run_query(enzyme_query)

# Merge the enzyme data with the main dataset containing chemistry, vital signs, and patient info
combined_df_with_enzymes = pd.merge(
    combined_df_with_chemistry.copy(),
    enzyme_df,
    on=['subject_id', 'hadm_id'],
    how='left'
)

# Display sample of the combined dataset
print(combined_df_with_enzymes.head())
print(f"Number of rows in the combined dataset with enzyme info: {combined_df_with_enzymes.shape[0]}")

/Users/zhuyu/anaconda3/envs/COMP90089/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
1    10006431  24638489 2129-01-23 23:36:00         508.0 2129-01-24 01:08:00   
2    10017531  22580355 2159-09-22 20:56:00        1164.0 2159-09-22 19:30:00   
3    10021357  25937617 2144-12-30 06:55:00        1249.0 2144-12-27 19:41:00   
4    10036086  28728587 2196-05-26 09:25:00         677.0 2196-05-20 02:47:00   

            dischtime gender  approximate_age_at_admission   race  actual_age  \
0 2159-03-06 16:51:00      F                            64  WHITE          64   
1 2129-01-30 16:50:00      F                            66  WHITE          67   
2 2159-10-24 13:40:00      M                            63  WHITE          64   
3 2145-01-04 19:54:00      F                            91  WHITE          91   
4 2196-06-12 11:42:00      M                            57  WHITE          58   

   ...  alp_enzyme  ast_en

In [97]:
# # 11. Get CRP info
# # Define the query to get CRP information for the first measurement after admission
# crp_query = f"""
# SELECT 
#     le.subject_id,
#     le.hadm_id,
#     MIN(le.charttime) AS first_charttime_crp,  -- Record the earliest chart time for CRP measurement
#     MAX(CASE WHEN le.itemid = 50889 AND le.valuenum > 0 THEN le.valuenum ELSE NULL END) AS crp_level
# FROM physionet-data.mimiciv_hosp.labevents AS le
# JOIN physionet-data.mimiciv_hosp.admissions AS adm
# ON le.subject_id = adm.subject_id AND le.hadm_id = adm.hadm_id
# WHERE le.hadm_id IS NOT NULL
# AND le.itemid = 50889  -- CRP item ID
# AND le.subject_id IN ({', '.join(map(str, subject_ids))})
# AND le.hadm_id IN ({', '.join(map(str, hadm_ids))})
# GROUP BY le.subject_id, le.hadm_id
# ORDER BY le.subject_id, le.hadm_id, first_charttime_crp
# """

# # Execute the CRP query
# crp_df = run_query(crp_query)

# # Merge the CRP data with the main dataset containing enzyme data, chemistry, vital signs, and patient info
# combined_df_with_crp = pd.merge(
#     combined_df_with_enzymes.copy(),
#     crp_df,
#     on=['subject_id', 'hadm_id'],
#     how='left'
# )

# # Display sample of the combined dataset with CRP data
# print(combined_df_with_crp.head())
# print(f"Number of rows in the combined dataset with CRP info: {combined_df_with_crp.shape[0]}")

In [116]:
# 12. Count ICU stays
# Define the query to count the number of ICU stays for each patient
icu_stay_query = f"""
SELECT DISTINCT i.subject_id, i.hadm_id, 
       CASE WHEN COUNT(i.stay_id) > 0 THEN 1 ELSE 0 END AS has_icu_stay
FROM `physionet-data.mimiciv_icu.icustays` AS i
JOIN `physionet-data.mimiciv_hosp.admissions` AS a
ON i.subject_id = a.subject_id
AND i.hadm_id = a.hadm_id
WHERE i.subject_id IN ({', '.join(map(str, combined_df_with_enzymes['subject_id'].unique()))})
AND i.hadm_id IN ({', '.join(map(str, combined_df_with_enzymes['hadm_id'].unique()))})
GROUP BY i.subject_id, i.hadm_id
"""

icu_stay_df = run_query(icu_stay_query)  

combined_df_with_icu_info = pd.merge(
    combined_df_with_enzymes.copy(),
    icu_stay_df[['subject_id', 'hadm_id', 'has_icu_stay']],
    on=['subject_id', 'hadm_id'],
    how='left'
)


combined_df_with_icu_info['has_icu_stay'].fillna(0, inplace=True)

print(combined_df_with_icu_info.head())
print(f"ICU stays indicated for {combined_df_with_icu_info['has_icu_stay'].sum()} records.")

/Users/zhuyu/anaconda3/envs/COMP90089/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
1    10006431  24638489 2129-01-23 23:36:00         508.0 2129-01-24 01:08:00   
2    10017531  22580355 2159-09-22 20:56:00        1164.0 2159-09-22 19:30:00   
3    10021357  25937617 2144-12-30 06:55:00        1249.0 2144-12-27 19:41:00   
4    10036086  28728587 2196-05-26 09:25:00         677.0 2196-05-20 02:47:00   

            dischtime gender  approximate_age_at_admission   race  actual_age  \
0 2159-03-06 16:51:00      F                            64  WHITE          64   
1 2129-01-30 16:50:00      F                            66  WHITE          67   
2 2159-10-24 13:40:00      M                            63  WHITE          64   
3 2145-01-04 19:54:00      F                            91  WHITE          91   
4 2196-06-12 11:42:00      M                            57  WHITE          58   

   ...  ast_enzyme  amylas

/var/folders/5w/c5jgggvd3f52shk667p25scc0000gn/T/ipykernel_63031/1092375779.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df_with_icu_info['has_icu_stay'].fillna(0, inplace=True)


In [117]:
final_dataset = combined_df_with_enzymes.copy()

In [118]:
# number of unique patients
num_unique_patients = final_dataset['subject_id'].nunique()
print(f"Number of unique patients in the final dataset: {num_unique_patients}")

Number of unique patients in the final dataset: 2104


In [119]:
nan_counts = final_dataset.isna().sum()
# Columns without NaN values
print("Columns without NaN values:")
print(final_dataset.columns[final_dataset.isna().sum() == 0])
# Columns with NaN values and their counts
nan_columns = nan_counts[nan_counts > 0]
print("Columns with NaN values and their counts:")
print(nan_columns)

Columns without NaN values:
Index(['subject_id', 'hadm_id', 'charttime', 'lipase_level', 'admittime',
       'dischtime', 'gender', 'approximate_age_at_admission', 'race',
       'actual_age', 'in_hospital_death', 'length_of_stay', 'is_confirmed_ap',
       'charlson_comorbidity_index'],
      dtype='object')
Columns with NaN values and their counts:
icd_code                      805
average_weight               1454
average_height               1713
first_charttime_vital        1453
heart_rate_vital             1453
sbp_vital                    1453
dbp_vital                    1453
mbp_vital                    1453
resp_rate_vital              1453
temperature_c_vital          1455
spo2_vital                   1453
first_charttime_chemistry      12
albumin_chemistry             618
globulin_chemistry           2234
total_protein_chemistry      2174
aniongap_chemistry             15
bicarbonate_chemistry          15
bun_chemistry                  14
calcium_chemistry              60
c

In [120]:
# Delete those rows with missing hadm_id
final_dataset = final_dataset.dropna(subset=['hadm_id'])

nan_counts = final_dataset.isna().sum()
# Columns without NaN values
print("Columns without NaN values:")
print(final_dataset.columns[final_dataset.isna().sum() == 0])
# Columns with NaN values and their counts
nan_columns = nan_counts[nan_counts > 0]
print("Columns with NaN values and their counts:")
print(nan_columns)

# Columns with nan counts(include 0)

Columns without NaN values:
Index(['subject_id', 'hadm_id', 'charttime', 'lipase_level', 'admittime',
       'dischtime', 'gender', 'approximate_age_at_admission', 'race',
       'actual_age', 'in_hospital_death', 'length_of_stay', 'is_confirmed_ap',
       'charlson_comorbidity_index'],
      dtype='object')
Columns with NaN values and their counts:
icd_code                      805
average_weight               1454
average_height               1713
first_charttime_vital        1453
heart_rate_vital             1453
sbp_vital                    1453
dbp_vital                    1453
mbp_vital                    1453
resp_rate_vital              1453
temperature_c_vital          1455
spo2_vital                   1453
first_charttime_chemistry      12
albumin_chemistry             618
globulin_chemistry           2234
total_protein_chemistry      2174
aniongap_chemistry             15
bicarbonate_chemistry          15
bun_chemistry                  14
calcium_chemistry              60
c

In [121]:
# Count how may patients have high amylase level by checking amylase_enzyme column > 1000
num_patients_high_amylase = final_dataset[final_dataset['amylase_enzyme'] > 1000]['subject_id'].nunique()
print(f"Number of unique patients with high amylase levels: {num_patients_high_amylase}")

# # Add a new column to the dataset to indicate high amylase levels
# combined_df_with_enzymes['is_high_amylase'] = combined_df_with_enzymes['amylase_enzyme'] > 1000

Number of unique patients with high amylase levels: 200


In [122]:
# # Generate a CSV file with the final combined dataset
# final_dataset.to_csv('AP_ICD_CCI_dataset.csv', index=False)

In [123]:
columns_to_keep = [
    'subject_id', 'hadm_id', 'charttime', 'lipase_level', 'admittime', 'dischtime', 
    'gender', 'approximate_age_at_admission', 'race', 'actual_age', 'in_hospital_death', 
    'length_of_stay', 'is_confirmed_ap', 'charlson_comorbidity_index', 
    'albumin_chemistry', 'creatinine_chemistry', 'aniongap_chemistry', 'bicarbonate_chemistry', 'bun_chemistry', 'calcium_chemistry', 'chloride_chemistry', 'glucose_chemistry', 'sodium_chemistry', 'potassium_chemistry', 'bilirubin_total_enzyme', 'amylase_enzyme', 'alt_enzyme', 'alp_enzyme', 'ast_enzyme'
]
df = final_dataset[columns_to_keep]
df['creatinine_albumin_ratio'] = df['creatinine_chemistry'] / df['albumin_chemistry']
# # Create ratio of CRP to albumin
# df['crp_albumin_ratio'] = df['crp_level'] / df['albumin_chemistry']
print(df.head())

   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
1    10006431  24638489 2129-01-23 23:36:00         508.0 2129-01-24 01:08:00   
2    10017531  22580355 2159-09-22 20:56:00        1164.0 2159-09-22 19:30:00   
3    10021357  25937617 2144-12-30 06:55:00        1249.0 2144-12-27 19:41:00   
4    10036086  28728587 2196-05-26 09:25:00         677.0 2196-05-20 02:47:00   

            dischtime gender  approximate_age_at_admission   race  actual_age  \
0 2159-03-06 16:51:00      F                            64  WHITE          64   
1 2129-01-30 16:50:00      F                            66  WHITE          67   
2 2159-10-24 13:40:00      M                            63  WHITE          64   
3 2145-01-04 19:54:00      F                            91  WHITE          91   
4 2196-06-12 11:42:00      M                            57  WHITE          58   

   ...  chloride_chemistry

/var/folders/5w/c5jgggvd3f52shk667p25scc0000gn/T/ipykernel_63031/1432187891.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['creatinine_albumin_ratio'] = df['creatinine_chemistry'] / df['albumin_chemistry']


In [124]:
# print the fial coloumns and count non nan values
print(df.info())
# number of unique patients
num_unique_patients = final_dataset['subject_id'].nunique()
print(f"Number of unique patients in the final dataset: {num_unique_patients}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2299 entries, 0 to 2298
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   subject_id                    2299 non-null   Int64         
 1   hadm_id                       2299 non-null   Int64         
 2   charttime                     2299 non-null   datetime64[us]
 3   lipase_level                  2299 non-null   float64       
 4   admittime                     2299 non-null   datetime64[us]
 5   dischtime                     2299 non-null   datetime64[us]
 6   gender                        2299 non-null   object        
 7   approximate_age_at_admission  2299 non-null   Int64         
 8   race                          2299 non-null   object        
 9   actual_age                    2299 non-null   Int64         
 10  in_hospital_death             2299 non-null   boolean       
 11  length_of_stay                

In [125]:
# Create csv file
df.to_csv('AP_ICD_CCI_dataset.csv', index=False)